<a href="https://colab.research.google.com/github/Nnamchisimi/Nnamchisimi/blob/main/NEW_STARORDER_FILE_VAUGCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***  BEST AND WORKS FINE FOR THE NEW STARORDER SITE, TRACKING LINK WORKS TOO OVERALL GOOD 20_01_25***

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (45).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (45).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Save df1_unique for verification
    df1_unique.to_excel(exdf1, index=False)

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Ensure DN no. is read as string to avoid scientific notation
df['DN no.'] = df['DN no.'].astype(str)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'back order'
    return 'No Dispatch Info'

df['status'] = df.apply(determine_status, axis=1)

# Merge tracking numbers into the DataFrame
tracking_dict = search_emails_for_tracking_numbers(df['DN no.'])
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()

Enter the customer info: Girne
Customer info Details:
       ±         gATP       Code Cust. No.  Cust. order no.  \
183  NaN        Green    No Code  CY822025         44051755   
184  NaN        Green    No Code  CY822025         44051755   
185  NaN        Green    No Code  CY822025         44051755   
186  NaN        Green    No Code  CY822025         44051755   
189  NaN          Red  Info Code  CY822025         44051755   
191  NaN        Green    No Code  CY822025         44051755   
196  NaN  YellowTruck    No Code  CY822025         44051755   
336  NaN       Yellow    No Code  CY822025         43931734   
453  NaN        Green    No Code  CY822025         43931734   
454  NaN        Green    No Code  CY822025         43931734   
455  NaN        Green    No Code  CY822025         43931734   
464  NaN        Green    No Code  CY822025         43931734   
478  NaN        Green    No Code  CY822025         43931734   
486  NaN        Green    No Code  CY822025         43931734   
4

('BYE', [b'LOGOUT Requested'])

***thıs gets the lınk , but needs to be ımproved ***

In [ ]:
import imaplib
import email
import re
import pandas as pd

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

# Function to fetch DN numbers and their corresponding STT link
def fetch_dn_and_stt_links(body):
    # Regular expression to match DN numbers (e.g., 6528016615)
    dn_numbers = re.findall(r'(?<=MBZ\s)(\d{10,})', body)

    # Regular expression to match STT numbers (e.g., 276701605763)
    stt_numbers = re.findall(r'(?<=STT Number\.:)\s*(\d+)', body)

    # Check if there are multiple STT numbers (in case of multiple tracking links)
    tracking_links = [f"https://eschenker.dbschenker.com/app/tracking-public/?refNumber={stt}" for stt in stt_numbers]

    # Associate each DN number with its corresponding tracking link (first tracking link for now)
    # If multiple STT numbers are found, we assume they correspond to the same tracking link
    dn_to_link = {dn: tracking_links[0] for dn in dn_numbers}

    return dn_to_link

# Function to search emails and extract DN numbers and their corresponding tracking links
def search_emails_for_dn_and_stt_links():
    mail.login(username, password)
    mail.select('inbox')

    # Search for all emails (you can adjust to a more specific query if needed)
    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return

    email_ids = data[0].split()
    print(f"Found {len(email_ids)} emails.")  # Debugging: check the number of emails fetched

    # Limit to the most recent 10 emails
    email_ids = email_ids[-10:]  # Get the last 10 email IDs

    # List to hold DN numbers and links to write to Excel
    dn_link_list = []

    # Process the most recent 10 emails
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    body = part.get_payload(decode=True).decode('utf-8', errors='replace')
                    break
        else:
            body = msg.get_payload(decode=True).decode('utf-8', errors='replace')

        # Fetch DN numbers and STT links from the email body
        dn_to_link = fetch_dn_and_stt_links(body)

        # Add the DN numbers and their links to the list
        for dn, link in dn_to_link.items():
            dn_link_list.append({"DN Number": dn, "Tracking Link": link})

    # Create a DataFrame from the list
    df = pd.DataFrame(dn_link_list)

    # Define the output file name
    output_file = "dn_tracking_links.xlsx"

    # Save the results to an Excel file
    df.to_excel(output_file, index=False)

    print(f"Output saved to {output_file}")

# Search for DN numbers and STT links in the most recent 10 emails
search_emails_for_dn_and_stt_links()

# Close the connection and logout
mail.close()
mail.logout()


Found 607 emails.
Output saved to dn_tracking_links.xlsx


('BYE', [b'LOGOUT Requested'])

**17_01_25 doesnt work but check **

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (42).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (42).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Limit the number of emails to search for performance optimization
MAX_EMAILS_TO_SEARCH = 50  # Adjust this to the number of emails you want to process

# Load and clean the first Excel file
print("Loading and cleaning the first Excel file...")
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    print("Cleaning 'Customer info' column...")
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]
print(f"Cleaned DataFrame size: {df_cleaned.shape}")

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)
print(f"Cleaned DataFrame saved to {file_path2}")

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
print("Loading additional Excel files...")
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
print("Checking for required columns in df1 and df2...")
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    print("Cleaning 'Part number' columns...")
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])
    df1_unique.to_excel(exdf1, index=False)
    print(f"df1_unique saved to {exdf1}")

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']], on=['Cleaned Customer info', 'Cleaned Part number'], how='left')
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
print("Loading updated DataFrame...")
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(username, password)

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

# Search emails efficiently
def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.select('inbox')

    # Using a more focused search query to speed things up
    result, data = mail.search(None, 'UNSEEN')  # or you can use 'SINCE' to get emails from a specific date
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    # Limit the number of emails to search
    email_ids = data[0].split()[:MAX_EMAILS_TO_SEARCH]  # Fetch only the most recent emails
    print(f"Total emails found: {len(email_ids)}")

    # Loop through each email and fetch headers first
    for email_id in email_ids:
        print(f"\nChecking email ID: {email_id}")
        result, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (FROM SUBJECT DATE)])')

        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Print the email header information (subject and from fields)
        print(f"Email Subject: {msg['Subject']}")
        print(f"Email From: {msg['From']}")

        # Check if any DN numbers are present in the subject or from fields
        for dn_number in dn_numbers:
            if dn_number in msg['Subject'] or dn_number in msg['From']:
                print(f"Found DN number {dn_number} in the email header.")
                # Now fetch the full email body (only if necessary)
                result, msg_data = mail.fetch(email_id, '(RFC822)')
                if result != 'OK':
                    print(f"Failed to fetch full email with ID {email_id}")
                    continue

                raw_email = msg_data[0][1]
                msg = email.message_from_bytes(raw_email)

                body = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        if part.get_content_type() == 'text/plain':
                            payload = part.get_payload(decode=True)
                            detected_encoding = chardet.detect(payload)['encoding']
                            body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                            break
                else:
                    payload = msg.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')

                # Check for the tracking number in the email body
                tracking_dict[dn_number] = fetch_stt_number(body)
                print(f"Tracking number for DN {dn_number}: {tracking_dict[dn_number]}")

    return tracking_dict

# Now call the function to search emails for tracking numbers and print the results
print("Searching for tracking numbers in emails...")
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Print final tracking numbers found
print("\nFinal tracking numbers found:")
for dn, tracking_number in tracking_dict.items():
    print(f"DN: {dn}, Tracking No: {tracking_number}")

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
print("Calculating expected stock entry date...")
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Save the final output to Excel
df.to_excel(output_file, index=False)
print(f"Final results saved to {output_file}")

# Closing the connection and logout
print("Closing the email connection...")
mail.close()
mail.logout()


Loading and cleaning the first Excel file...
Cleaning 'Customer info' column...
Cleaned DataFrame size: (3071, 63)
Cleaned DataFrame saved to /content/niv_StarOrderplus_Order_items_new.xlsx
Enter the customer info: Girne
Customer info Details:
       ±    gATP  Code Cust. No.  Cust. order no.     Part number  \
130   ⊞   Green         CY822025         44051755  A 270 159 0600   
131   ⊞   Green         CY822025         44051755  A 004 159 5803   
132   ⊞   Green         CY822025         44051755  A 271 016 1221   
133   ⊞   Green         CY822025         44051755  A 271 016 1321   
136   ⊞     Red   C21   CY822025         44051755  A 004 994 1845   
...   ..     ...   ...       ...              ...             ...   
3126  ⊞   Green         CY822025         43281568  A 223 835 2300   
4034  ⊞   Green         CY822025         43041533  A 270 200 0200   
4043  ⊞   Green         CY822025         43041533  A 222 899 0600   
4051  ⊞   Green         CY822025         43041533  A 447 880 1114 

('BYE', [b'LOGOUT Requested'])

In [ ]:
import re

expression = re.compile("([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

emails= ["test@hotmail.co.uk", "test@", "test!", "test@live.com"]

for i in emails:
    matches = expression.match(i)
    if (not matches):
        print(i, "INVALID EMAIL")
    else:
        print(i, "is a valid email!")

test@hotmail.co.uk is a valid email!
test@ INVALID EMAIL
test! INVALID EMAIL
test@live.com is a valid email!
